In [36]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

tfk = tf.keras
tfkl = tf.keras.layers

In [37]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [38]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Importing data

In [39]:
# Dataset folders 
dataset_dir = '/kaggle/input/an2dl-challenge-1-nt/data_splitted_no_test'
training_dir = os.path.join(dataset_dir, 'train')
validation_dir = os.path.join(dataset_dir, 'val')

In [40]:
# Images are divided into folders, one for each class. 
# If the images are organized in such a way, we can exploit the 
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator, and for the trainign with Data Augmentation
train_data_gen = ImageDataGenerator(rotation_range=30, 
                                    height_shift_range=0.2, 
                                    width_shift_range=0.2, 
                                    zoom_range=0.2, 
                                    horizontal_flip=True, 
                                    shear_range=0.2, 
                                    fill_mode='reflect',
                                    rescale=1/255.) # rescale value is multiplied to the image ############################################
valid_data_gen = ImageDataGenerator(rescale=1/255.)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                                       target_size=(96,96),
                                                       color_mode='rgb',
                                                       classes=None, # can be set to labels
                                                       class_mode='categorical',
                                                       batch_size=8,
                                                       shuffle=True,
                                                       seed=seed)

valid_gen = valid_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=False,
                                               seed=seed)

Found 3648 images belonging to 8 classes.
Found 648 images belonging to 8 classes.


## Model

In [41]:
import os
os.listdir("/kaggle/input/ir-v104/IR_v1.0.4")

['variables', 'saved_model.pb', 'keras_metadata.pb']

In [42]:
# Re-load the model after transfer learning
IR = tfk.models.load_model('/kaggle/input/ir-v104/IR_v1.0.4')
IR.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 96, 96, 3)]       0         
_________________________________________________________________
inception_resnet_v2 (Functio (None, 1, 1, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1536)              0         
_________________________________________________________________
Classifier1 (Dense)          (None, 512)               786944    
_________________________________________________________________
activation_203 (Activation)  (None, 512)               0         
_________________________________________________________________
batch_normalization_203 (Bat (None, 512)               2048  

In [43]:
IR.trainable = False

In [46]:
IR_2 = tfk.models.Sequential(IR.layers[:-11])

In [47]:
IR_2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 1, 1, 1536)        54336736  
Total params: 54,336,736
Trainable params: 0
Non-trainable params: 54,336,736
_________________________________________________________________


In [48]:
input_shape = (96, 96, 3)
epochs = 200

In [49]:
def inceptionresnetv2_2(input_shape):

    # Build the neural network layer by layer
    input_layer_2 = tfkl.Input(shape=input_shape, name='input_layer')
    
    #x = tfkl.Resizing(64, 64, interpolation="bicubic", name='resizing')(input_layer)
    ir_2 = IR_2(input_layer_2)

    gap = tfkl.GlobalAveragePooling2D()(ir_2)
    drop1 = tfkl.Dropout(0.4, seed=seed)(gap)
    classifier_layer1 = tfkl.Dense(units=512, name='Classifier1', kernel_initializer=tfk.initializers.HeUniform(seed), activation='relu')(drop1)
    drop2 = tfkl.Dropout(0.4, seed=seed)(classifier_layer1)
    output_layer = tfkl.Dense(units=8, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='output_layer')(drop2)
    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer_2, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.RMSprop(learning_rate=0.001), metrics='accuracy')

    # Return the model
    return model

In [50]:
# Build model
IR_final = inceptionresnetv2_2(input_shape)
IR_final.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 96, 96, 3)]       0         
_________________________________________________________________
sequential_5 (Sequential)    (None, 1, 1, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1536)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 1536)              0         
_________________________________________________________________
Classifier1 (Dense)          (None, 512)               786944    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
output_layer (Dense)         (None, 8)                 4104  

In [ ]:
# Train the model
IR_history = IR_final.fit(
    x = train_gen,
    batch_size = 256,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)],
).history

In [ ]:
# Save the best model
IR_final.save('IR_v1.1.3')

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(IR_history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(IR_history['val_loss'], label='Standard', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)
plt.savefig('IR_v1.1.3_loss.png')

plt.figure(figsize=(15,5))
plt.plot(IR_history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(IR_history['val_accuracy'], label='Standard', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)
plt.savefig('IR_v1.1.3_acc.png')

plt.show()

In [ ]:
# Evaluate on val
val_metrics = IR_final.evaluate(valid_gen, return_dict=True)

print()
print("Val metrics")
print(val_metrics)